In [1]:
from deap import base
from deap import creator
from deap import tools
from deap import algorithms

import random
import numpy

import matplotlib.pyplot as plt

# problem constants:
ONE_MAX_LENGTH = 100  # length of bit string to be optimized

# Genetic Algorithm constants:
POPULATION_SIZE = 500
P_CROSSOVER = 0.7  # probability for crossover
P_MUTATION = 0.01   # probability for mutating an individual
MAX_GENERATIONS = 50
HALL_OF_FAME_SIZE = 10

N_RUNS = 300
NBR_ITEMS =100
MAX_WEIGHT = 1000
# set the random seed:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
# Create the item dictionary: item name is an integer, and value is 
# a (value, weight) 2-uple.
items = {}
# Create random items and store them in the items' dictionary.
for i in range(NBR_ITEMS):
    items[i] = (random.randint(1, 10), random.randint(1, 100))

# set the random seed:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

toolbox = base.Toolbox()

# create an operator that randomly returns 0 or 1:
toolbox.register("zeroOrOne", random.randint, 0, 1)

# define a single objective, maximizing fitness strategy:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

# create the Individual class based on list:
creator.create("Individual", list, fitness=creator.FitnessMax)
#creator.create("Individual", array.array, typecode='b', fitness=creator.FitnessMax)

# create the individual operator to fill up an Individual instance:
toolbox.register("individualCreator", tools.initRepeat, creator.Individual, toolbox.zeroOrOne, NBR_ITEMS)

# create the population operator to generate a list of individuals:
toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)




# fitness calculation:
# compute the number of '1's in the individual
def oneMaxFitness(individual):
    return sum(individual),  # return a tuple



def evalKnapsack2(individual):
    weight = 0.0
    value = 0.0
    for i in range(NBR_ITEMS):
        value += items[i][0]*individual[i]
        weight += items[i][1]*individual[i]
    if  weight > MAX_WEIGHT:
        return (value-(weight-MAX_WEIGHT)*2),weight
    else:             
        return (value), weight

toolbox.register("evaluate", evalKnapsack2)

# genetic operators:

# Tournament selection with tournament size of 3:
#toolbox.register("select", tools.selRoulette)

# Tournament selection with tournament size of 3:
toolbox.register("select", tools.selTournament, tournsize=3)

# Single-point crossover:
toolbox.register("mate", tools.cxOnePoint)

# Flip-bit mutation:
# indpb: Independent probability for each attribute to be flipped
toolbox.register("mutate", tools.mutFlipBit, indpb=1.0/ONE_MAX_LENGTH)



# Genetic Algorithm flow:
def main():
    maxList = []
    avgList = []
    minList = []
    stdList = []
    for r in range(0, N_RUNS):
        # create initial population (generation 0):
        population = toolbox.populationCreator(n=POPULATION_SIZE)

        # define the hall-of-fame object:
        hof = tools.HallOfFame(HALL_OF_FAME_SIZE)

        # prepare the statistics object:
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("avg", numpy.mean)
        stats.register("std", numpy.std)
        stats.register("min", numpy.min)
        stats.register("max", numpy.max)

        # perform the Genetic Algorithm flow:
        population, logbook = algorithms.eaSimple(population, toolbox, cxpb=P_CROSSOVER, mutpb=P_MUTATION,
                                                   ngen=MAX_GENERATIONS,
                                                  stats=stats, halloffame=hof, verbose=True)

        # print Hall of Fame info:
        print("Hall of Fame Individuals = ", *hof.items, sep="\n")
        print("\nXXBest Ever Individual = ", hof.items[0], "\nXXFitness: ", r ," ",evalKnapsack2(hof.items[0]))
     

        # Genetic Algorithm is done with this run - extract statistics:
        meanFitnessValues, stdFitnessValues, minFitnessValues, maxFitnessValues  = logbook.select("avg", "std", "min", "max")
        
        # Save statistics for this run:
        avgList.append(meanFitnessValues)
        stdList.append(stdFitnessValues)
        minList.append(minFitnessValues)
        maxList.append(maxFitnessValues)


    # Genetic Algorithm is done (all runs) - plot statistics:
    x = numpy.arange(0, MAX_GENERATIONS+1)
    avgArray = numpy.array(avgList)
    stdArray = numpy.array(stdList)
    minArray = numpy.array(minList)
    maxArray = numpy.array(maxList)
    plt.xlabel('Generation')
    plt.ylabel('Fitness')
    plt.title('Max and Average Fitness for Knapsack with Tournament Wheel Selection')
    plt.errorbar(x, avgArray.mean(0), yerr=stdArray.mean(0),label="Average",color="Red")
    plt.errorbar(x, maxArray.mean(0), yerr=maxArray.std(0),label="Best", color="Green")
    plt.show()
    
    
if __name__ == '__main__':
    main()


In D:\Anaconda\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Anaconda\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Anaconda\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In D:\Anaconda\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In D:\Anaconda\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.jpeg_quality rcparam was deprecated in Matplo

gen	nevals	avg     	std    	min  	max  
0  	500   	-3013.69	581.548	-4842	-1206
1  	336   	-2538.53	499.026	-3824	-1042
2  	335   	-2120.65	452.271	-3636	-503 
3  	360   	-1745.24	393.757	-2839	-430 
4  	366   	-1406.3 	360.859	-2942	-402 
5  	335   	-1070.11	324.477	-2019	-183 
6  	356   	-781.454	286.063	-1833	117  
7  	344   	-528.23 	246.356	-1503	168  
8  	375   	-325.852	239.864	-1255	179  
9  	390   	-131.634	210.141	-952 	179  
10 	360   	25.734  	148.458	-745 	185  
11 	332   	114.986 	75.1407	-254 	204  
12 	362   	138.082 	64.8822	-363 	204  
13 	327   	151.928 	58.686 	-309 	204  
14 	349   	165.226 	47.3589	-261 	204  
15 	337   	178.32  	37.9195	-261 	210  
16 	355   	185.48  	34.6304	-250 	213  
17 	341   	188.83  	39.6842	-104 	218  
18 	337   	195.568 	36.2519	-224 	218  
19 	354   	200.512 	28.5995	-31  	221  
20 	346   	203.272 	29.3074	-14  	222  
21 	358   	207.254 	23.3022	-23  	226  
22 	356   	209.326 	23.2093	14   	226  
23 	366   	206.422 	41.2871	-156 	231  


gen	nevals	avg     	std    	min  	max  
0  	500   	-3000.11	598.732	-4914	-1394
1  	349   	-2530.67	476.186	-3927	-1191
2  	353   	-2115.9 	388.811	-3363	-906 
3  	366   	-1773.3 	388.136	-2947	-403 
4  	344   	-1444.96	348.877	-2690	-403 
5  	346   	-1128.45	326.259	-2121	-151 
6  	355   	-840.558	302.343	-1752	134  
7  	349   	-575.584	322.591	-1755	156  
8  	349   	-312.252	282.496	-1093	163  
9  	337   	-68.564 	195.426	-756 	173  
10 	332   	67.586  	113.076	-502 	173  
11 	384   	106.468 	86.9237	-613 	165  
12 	356   	125.204 	62.5752	-334 	165  
13 	377   	125.078 	79.3864	-498 	173  
14 	359   	130.556 	79.4689	-483 	172  
15 	370   	141.926 	55.1241	-249 	175  
16 	351   	145.792 	54.0518	-248 	192  
17 	357   	147.272 	64.2539	-465 	192  
18 	357   	154.732 	53.1401	-261 	199  
19 	364   	158.926 	45.6821	-249 	204  
20 	361   	159.728 	47.6828	-149 	205  
21 	339   	164.912 	47.7669	-229 	205  
22 	366   	164.106 	59.9146	-248 	208  
23 	378   	174.306 	44.9677	-202 	210  


gen	nevals	avg     	std    	min  	max  
0  	500   	-3057.86	596.983	-4926	-1368
1  	362   	-2535.42	488.372	-3863	-1109
2  	369   	-2111.41	392.979	-3623	-1007
3  	359   	-1776.34	347.091	-2776	-311 
4  	336   	-1471.2 	351.929	-2573	-276 
5  	368   	-1175.3 	352.983	-2103	-63  
6  	351   	-870.452	324.494	-1825	-63  
7  	351   	-599.808	307.393	-1511	87   
8  	346   	-327.188	209.202	-1087	151  
9  	351   	-159.554	153.68 	-604 	157  
10 	362   	-40.846 	140.883	-667 	168  
11 	349   	54.7    	104.515	-353 	163  
12 	319   	113.492 	62.1704	-424 	178  
13 	363   	130.154 	49.9105	-250 	167  
14 	374   	139.152 	42.983 	-214 	183  
15 	347   	145.556 	37.9302	-130 	189  
16 	347   	149.632 	43.4744	-172 	193  
17 	326   	155.91  	38.496 	-118 	197  
18 	361   	157.438 	47.9739	-350 	197  
19 	342   	165.952 	44.2433	-146 	202  
20 	360   	170.292 	43.8021	-278 	207  
21 	361   	179.116 	33.7687	-103 	207  
22 	336   	187.89  	24.653 	10   	208  
23 	360   	191.828 	24.5256	-39  	211  


gen	nevals	avg     	std    	min  	max  
0  	500   	-2985.89	578.792	-4471	-1290
1  	357   	-2469.95	442.888	-3760	-1214
2  	372   	-2079.55	367.753	-3097	-735 
3  	363   	-1752.53	362.024	-2764	-597 
4  	347   	-1445.4 	327.448	-2401	-597 
5  	352   	-1179.65	318.486	-2209	-278 
6  	343   	-904.682	270.321	-1731	-90  
7  	351   	-669.634	222.874	-1345	10   
8  	350   	-479.824	193.826	-966 	92   
9  	375   	-307.034	193.778	-932 	164  
10 	347   	-148.728	181.128	-730 	175  
11 	350   	-9.834  	156.866	-581 	189  
12 	372   	96.35   	97.631 	-475 	186  
13 	359   	142.204 	56.5024	-179 	193  
14 	362   	156.552 	49.2601	-198 	203  
15 	363   	166.392 	37.4273	-170 	206  
16 	360   	173.274 	30.151 	-57  	207  
17 	361   	179.01  	29.8963	-111 	208  
18 	378   	183.278 	28.6463	-21  	210  
19 	345   	191.064 	22.3799	0    	215  
20 	357   	194.106 	34.2062	-192 	220  
21 	347   	196.478 	30.295 	-71  	222  
22 	377   	199.126 	29.2746	7    	229  
23 	328   	206.828 	22.8887	8    	236  


...

gen	nevals	avg     	std    	min  	max  
0  	500   	-2984.41	582.573	-4602	-1010
1  	354   	-2526.81	465.366	-3712	-1001
2  	357   	-2117.98	437.119	-3245	-901 
3  	360   	-1738.72	386.319	-3045	-566 
4  	343   	-1393.75	350.867	-2380	-38  
5  	375   	-1081.79	342.32 	-2305	143  
6  	352   	-791.086	325.168	-2009	149  
7  	348   	-496.028	279.251	-1376	165  
8  	336   	-267.254	260.822	-1122	172  
9  	367   	-75.06  	204.66 	-794 	178  
10 	333   	74.98   	122.475	-548 	178  
11 	348   	131.092 	56.5689	-298 	177  
12 	354   	133.258 	72.5625	-606 	183  
13 	357   	148.452 	47.3335	-434 	183  
14 	343   	151.036 	63.6432	-361 	186  
15 	343   	158.288 	44.7522	-369 	191  
16 	346   	165.372 	42.3395	-422 	192  
17 	327   	171.856 	35.1946	-384 	202  
18 	355   	175.21  	37.803 	-455 	203  
19 	373   	179.016 	38.0876	-362 	214  
20 	343   	186.756 	26.1966	-69  	214  
21 	349   	190.398 	31.7477	-174 	216  
22 	366   	194.638 	29.2117	-184 	216  
23 	344   	200.2   	30.4333	-114 	216  


gen	nevals	avg    	std    	min  	max  
0  	500   	-2939.5	589.676	-5156	-1071
1  	327   	-2496.83	413.495	-3782	-1071
2  	358   	-2155.82	384.704	-3421	-719 
3  	337   	-1817.61	388.389	-2939	-324 
4  	347   	-1464.59	388.742	-2569	-64  
5  	370   	-1142.14	357.538	-2026	-124 
6  	350   	-821.352	320.01 	-1753	21   
7  	338   	-536.762	290.262	-1395	147  
8  	316   	-280.706	255.96 	-1216	174  
9  	351   	-73.202 	188.145	-926 	174  
10 	355   	55.134  	109.63 	-393 	174  
11 	335   	109.54  	66.3037	-221 	174  
12 	374   	129.01  	47.7924	-209 	180  
13 	333   	134.576 	50.2691	-232 	184  
14 	349   	145.134 	45.1135	-255 	191  
15 	347   	152.84  	40.5901	-135 	193  
16 	334   	161.324 	34.8863	-159 	204  
17 	367   	161.638 	42.1691	-131 	207  
18 	335   	169.462 	35.1793	-143 	204  
19 	360   	171.046 	39.6167	-177 	210  
20 	367   	176.682 	40.9054	-94  	210  
21 	365   	178.912 	40.2869	-62  	214  
22 	346   	184.654 	36.5678	-58  	214  
23 	347   	191.756 	32.46  	-81  	217  
24

gen	nevals	avg     	std    	min  	max  
0  	500   	-2946.48	561.045	-4360	-1411
1  	330   	-2475   	454.217	-3716	-914 
2  	322   	-2081.18	391.963	-3658	-914 
3  	348   	-1752.46	373.903	-2930	-743 
4  	380   	-1425.24	320.023	-2179	-475 
5  	341   	-1151   	303.409	-2366	-109 
6  	365   	-908.992	296.023	-1870	-58  
7  	364   	-644.896	269.153	-1810	163  
8  	366   	-407.814	260.8  	-1352	202  
9  	372   	-198.512	232.182	-889 	183  
10 	358   	-4.056  	180.151	-617 	189  
11 	327   	116.768 	97.5075	-512 	198  
12 	344   	158.528 	44.2776	-131 	200  
13 	324   	174.15  	28.6899	-148 	200  
14 	333   	176.634 	33.0102	-108 	203  
15 	344   	183.822 	31.2882	-99  	206  
16 	337   	183.542 	36.6083	-179 	207  
17 	368   	181.858 	54.138 	-244 	213  
18 	376   	186.39  	50.2036	-239 	214  
19 	370   	195.714 	28.1126	-12  	218  
20 	327   	201.6   	21.4298	-12  	218  
21 	359   	202.048 	32.554 	-107 	229  
22 	351   	203.582 	37.722 	-305 	224  
23 	341   	207.904 	28.8734	-149 	231  


gen	nevals	avg     	std    	min  	max 
0  	500   	-2978.79	634.474	-4797	-799
1  	342   	-2445.44	516.832	-4249	-617
2  	345   	-2033.34	452.91 	-3374	-64 
3  	371   	-1680   	373.593	-2754	-121
4  	351   	-1345.73	347.241	-2466	2   
5  	340   	-1065.82	296.454	-2077	61  
6  	337   	-807.898	265.906	-1620	140 
7  	345   	-583.858	263.762	-1504	147 
8  	324   	-376.626	261.435	-1155	149 
9  	348   	-146.37 	211.966	-849 	157 
10 	356   	15.874  	152.595	-737 	167 
11 	320   	95.604  	94.3185	-530 	168 
12 	367   	119.74  	57.8832	-383 	168 
13 	387   	126.056 	63.59  	-404 	174 
14 	355   	136.286 	50.3517	-185 	177 
15 	369   	145.7   	43.6346	-200 	186 
16 	362   	149.652 	48.8398	-237 	188 
17 	327   	159.202 	36.3911	-192 	194 
18 	340   	159.636 	48.848 	-189 	199 
19 	353   	164.836 	43.5846	-166 	202 
20 	326   	170.756 	40.5175	-112 	213 
21 	350   	176.576 	34.5632	-65  	213 
22 	344   	174.748 	45.1935	-141 	222 
23 	362   	175.338 	48.927 	-136 	213 
24 	335   	183.762 	41.49

KeyboardInterrupt: 